In [1]:
import module_loader
import pandas as pd
from bookirds.curves import *
from bookirds.dual import Dual

# Trade Directionality Hedging

### Add a short historical data sample for covariance

In [2]:
portfolio = Portfolio()
historical_rates = pd.DataFrame({
    "2Y": [1.199, 1.228, 1.210, 1.215, 1.203, 1.159, 1.175, 1.188, 1.159, 1.100],
    "5Y": [1.663, 1.696, 1.665, 1.680, 1.677, 1.657, 1.673, 1.676, 1.653, 1.600],
    "10Y": [1.928, 1.945, 1.934, 1.93, 1.934, 1.931, 1.958, 1.972, 1.932, 1.900],
    "30Y": [2.201, 2.217, 2.228, 2.239, 2.226, 2.235, 2.242, 2.236, 2.22, 2.200],
})
historical_chgs = historical_rates.diff(-1)*100
Q = historical_chgs.cov().to_numpy()
Q

array([[8.66      , 7.3775    , 5.13625   , 2.185     ],
       [7.3775    , 7.3075    , 4.73      , 2.1575    ],
       [5.13625   , 4.73      , 4.89111111, 1.76236111],
       [2.185     , 2.1575    , 1.76236111, 1.86111111]])

### PCA Approach

In [3]:
lambd, E = portfolio.pca(Q)
S_ini = np.array([-1, 2.0, -1, 0])[:, np.newaxis]
portfolio.pca_hedge_adjustment(None, Q, S_ini)

array([[-0.06356746],
       [-0.05827074],
       [-0.04329331],
       [-0.01923775]])

The problem with the above minimal adjustment is that it includes a 30Y bucket even though 30Y was not considered in the original trade. We can explicitly set that bucket to be zero in our directionality adjustment.

In [4]:
x = portfolio.pca_hedge_adjustment(None, Q, S_ini, L=[3])
x

array([[-6.60942303e-02],
       [-6.05869633e-02],
       [-4.50141914e-02],
       [ 1.05803249e-19]])

In [5]:
S = (S_ini + x)
S

array([[-1.06609423e+00],
       [ 1.93941304e+00],
       [-1.04501419e+00],
       [ 1.05803249e-19]])

In [6]:
delta_r = np.array([17.2, 18.1, 12.5, 8.1])[:, np.newaxis]
print("PnL: ", np.matmul(S.T, delta_r))

PnL:  [[3.70387781]]


Below we can calculate the non centralised PC multipliers and explain the PnL from PCA risks and multipliers. The PnL value is invariant.

In [7]:
non_cent_multipliers = np.matmul(delta_r.T, E).T
pca_risk = np.matmul(E.T, S)
print(non_cent_multipliers, "\n", pca_risk)

[[-28.91591188]
 [ -1.39281418]
 [  2.40507236]
 [  1.20653883]] 
 [[-5.55318383e-17]
 [ 8.19132846e-01]
 [ 9.63138609e-01]
 [ 2.09554764e+00]]


In [8]:
print("PnL from PCA: ", np.matmul(pca_risk.T, non_cent_multipliers))

PnL from PCA:  [[3.70387781]]


Since the risk to PC1 is zero we can determine the residuals if we exclude every PC except the first

In [9]:
nc_multiplier_neg = non_cent_multipliers.copy()
nc_multiplier_neg[1,0] = 0
nc_multiplier_neg[2,0] = 0
nc_multiplier_neg[3,0] = 0

In [10]:
delta_r_adj = np.matmul(E, nc_multiplier_neg)
delta_r_adj

array([[18.68178329],
       [17.1251335 ],
       [12.72343084],
       [ 5.65376358]])

In [11]:
residuals = delta_r - delta_r_adj
residuals

array([[-1.48178329],
       [ 0.9748665 ],
       [-0.22343084],
       [ 2.44623642]])

The PnL from residuals is invariant since PC1 risk was explcitly set to be zero.

In [12]:
print("PnL from residuals: ", np.matmul(S.T, residuals))

PnL from residuals:  [[3.70387781]]


### CoVaR Minimisation Approach

In [13]:
S_ini1 = np.array([-1, 2, -1.0, 0])[:, np.newaxis]
S_ini2 = np.array([-1, 2, -1.0, 0])[:, np.newaxis]

In [14]:
x1 = portfolio.covar_mmt(None, Q, [1], S_ini1)
x2 = portfolio.covar_mmt(None, Q, [0,2], S_ini2)

In [15]:
S1 = (S_ini1 + x1) 
S1 * 1.9394 / S1[1,0]

array([[-1.17052781],
       [ 1.9394    ],
       [-1.17052781],
       [ 0.        ]])

In [16]:
S2 = (S_ini2 + x2)
S2 * 1.9394 / S2[1,0]

array([[-1.43121075],
       [ 1.9394    ],
       [-0.37257501],
       [ 0.        ]])

### Multivariate Least Squares Regression

In [17]:
delta_r = historical_chgs.iloc[:9,:].to_numpy()
delta_r = delta_r - delta_r.mean(axis=0)
delta_r

array([[-4.        , -4.        , -2.01111111, -1.61111111],
       [ 0.7       ,  2.4       ,  0.78888889, -1.11111111],
       [-1.6       , -2.2       ,  0.08888889, -1.11111111],
       [ 0.1       , -0.4       , -0.71111111,  1.28888889],
       [ 3.3       ,  1.3       , -0.01111111, -0.91111111],
       [-2.7       , -2.3       , -3.01111111, -0.71111111],
       [-2.4       , -1.        , -1.71111111,  0.58888889],
       [ 1.8       ,  1.6       ,  3.68888889,  1.58888889],
       [ 4.8       ,  4.6       ,  2.88888889,  1.98888889]])

Assume a 2s5s10s trade hedged for 5Y only

In [18]:
S_ini = np.array([-1, 2.0, -1, 0])[:, np.newaxis]
y = np.matmul(delta_r, S_ini)
x_1 = np.array([0, 1.0, 0, 0])[:, np.newaxis]
X = np.matmul(delta_r, x_1)

In [19]:
beta = np.matmul(1/np.matmul(X.T, X), np.matmul(X.T, y))

In [20]:
S_trade = -np.matmul(x_1, beta)
S_trade

array([[-0.        ],
       [-0.34314061],
       [-0.        ],
       [-0.        ]])

In [21]:
S = S_ini + S_trade
S * 1.9394 / S[1,0]

array([[-1.17052781],
       [ 1.9394    ],
       [-1.17052781],
       [ 0.        ]])

This result is the same as the CoVaR minimisation against the single 5Y instrument

Now assume a hedge to 2y and 10y.

In [22]:
x_j = np.array([
    [1.0, 0],
    [0, 0],
    [0, 1.0],
    [0, 0]
])
X = np.matmul(delta_r, x_j)
beta = np.linalg.solve(np.matmul(X.T, X), np.matmul(X.T, y))
S_trade = - np.matmul(x_j, beta)
S = S_ini + S_trade
S * 1.9394 / S[1,0]

array([[-1.43121075],
       [ 1.9394    ],
       [-0.37257501],
       [ 0.        ]])

This result is the same as the multi-instrument VaR minimisation strategy measured over 2Y and 10Y.

Now we will try to hedge against the first PC

In [23]:
x_1 = E[:, [0]]
x_1[3, 0] = 0
X = np.matmul(delta_r, x_1)
beta = np.linalg.solve(np.matmul(X.T, X), np.matmul(X.T, y))
S_trade = - np.matmul(x_1, beta)
S = S_ini + S_trade
S
# S * 1.93941304 / S[1,0]

array([[-1.06600256],
       [ 1.93949707],
       [-1.04495176],
       [ 0.        ]])

In [24]:
pca_risk = np.matmul(E.T, S)
pca_risk

array([[-1.36462561e-04],
       [ 8.19146441e-01],
       [ 9.63115634e-01],
       [ 2.09555288e+00]])

This result is very close to the PCA minimisation approach and we see above that risk to PC1 is essentially fully hedged.